## TODO

Try flow MNIST with UNet MLP, UNet / DiT

In [ ]:
import math
import numpy as np
import torch
import torch.utils.checkpoint
import torchvision
import matplotlib.pyplot as plt

from torchvision import transforms
from tqdm.auto import tqdm

from rectified_flow.rectified_flow import RectifiedFlow
from rectified_flow.utils import match_dim_with_data

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.backends.cuda.matmul.allow_tf32 = True
torch.backends.cudnn.allow_tf32 = True

## Load dataset

In [ ]:
batch_size = 512

transform_list = [
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,)),
]
train_dataset = torchvision.datasets.MNIST(
    root="./data", train=True, download=True, transform=transforms.Compose(transform_list)
)

train_dataloader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size=batch_size,
    shuffle=True,
    num_workers=0,          
    pin_memory=True,
    persistent_workers=False,
)

batch = next(iter(train_dataloader))
print(batch[0].shape)  # torch.Size([256, 1, 28, 28])

## Train Unconditional Generation

In [ ]:
model_type = "unet"    # "mlp" or "unet" or "dit"
from rectified_flow.models.enhanced_mlp import VarMLP
from rectified_flow.models.utils import EMAModel

if model_type == "unet":
    from rectified_flow.models.unet import SongUNet, SongUNetConfig
    config = SongUNetConfig(
        img_resolution = 28,
        in_channels = 1,                  # Number of color channels at input.
        out_channels = 1,                 # Number of color channels at output.
        label_dim = 0,                    # Number of class labels, 0 = unconditional.
        augment_dim = 0,                  # Augmentation label dimensionality, 0 = no augmentation.

        model_channels = 64,                # Base multiplier for the number of channels.
        channel_mult = [2, 2],              # Channel multipliers for each resolution.
        channel_mult_emb = 2,               # Multiplier for the dimensionality of the embedding vector.
        num_blocks = 2,                     # Number of residual blocks per resolution.
        attn_resolutions = [14],            # Resolutions at which to apply attention.
        dropout = 0.13,                     # Dropout probability of intermediate activations.
        label_dropout = 0.0,                # Dropout probability of class labels for classifier-free guidance.
        embedding_type = "positional",      # Timestep embedding type: 'positional' or 'fourier'.
        channel_mult_time = 1,              # Timestep embedding size: 1 for DDPM++, 2 for NCSN++.
        encoder_type = "standard",          # Encoder architecture: 'standard' or 'residual'.
        decoder_type = "standard",          # Decoder architecture: 'standard' or 'residual'.
        resample_filter = [1, 1]
    )
    flow_model = SongUNet(config)
    data_shape = (1, 28, 28)
elif model_type == "dit":
    raise NotImplementedError
else:
    raise ValueError(f"Unknown model type: {model_type}")

flow_model = flow_model.to(device)

print(f"Number of parameters in flow model: {sum(p.numel() for p in flow_model.parameters() if p.requires_grad):,}")

ema_flow = EMAModel(flow_model, ema_halflife_kimg=1.0, ema_rampup_ratio=0.05)

optimizer = torch.optim.AdamW(
    flow_model.parameters(),
    lr=5e-4, weight_decay=0.01,
    betas=(0.9, 0.95)
)

compiled_flow = torch.compile(flow_model, mode="reduce-overhead", fullgraph=False, dynamic=False)

compiled_flow.train()
optimizer.zero_grad()

In [ ]:
epoch = 200
cur_nimg = 0

try:
    from tqdm.auto import tqdm
except Exception:
    from tqdm import tqdm

def safe_tqdm_write(msg: str):
    try:
        tqdm.write(msg)
    except Exception:
        print(msg)

zero_to_none = True
grad_clip_norm = None

global_step = 0
running_loss = None
smooth_alpha = 0.99

# test model
with torch.no_grad():
    batch = next(iter(train_dataloader))
    x_1, c = batch
    x_1 = x_1.to(device, non_blocking=True).reshape(x_1.shape[0], *data_shape)
    x_0 = torch.randn_like(x_1)
    t = torch.rand(x_1.shape[0], device=x_1.device)
    v_pred = flow_model(x_1, t)
    print(x_1.shape, x_0.shape, v_pred.shape)
    sq_err = (v_pred - (x_1 - x_0)).pow(2).sum(dim=1)
    loss = sq_err.mean()
    safe_tqdm_write(f"Initial test loss: {loss.item():.4f}")

In [ ]:
for ep in tqdm(range(epoch), desc="Epochs", position=0):
    compiled_flow.train()
    pbar = tqdm(train_dataloader, desc=f"ep {ep+1}/{epoch}", leave=False, position=1)
    for step, batch in enumerate(pbar):
        optimizer.zero_grad(set_to_none=zero_to_none)

        x_1, c = batch
        x_1 = x_1.to(device, non_blocking=True).reshape(x_1.shape[0], *data_shape)
        x_0 = torch.randn_like(x_1)

        B = x_1.shape[0]
        t = torch.rand(B, device=x_1.device, dtype=x_1.dtype)
        t_b = t.view(B, 1, 1, 1)

        with torch.autocast(device_type="cuda", dtype=torch.bfloat16):
            x_t = t_b * x_1 + (1 - t_b) * x_0
            dot_x_t = x_1 - x_0
            v_pred = compiled_flow(x_t, t)

        loss = torch.nn.functional.mse_loss(v_pred.float(), dot_x_t.float())

        loss.backward()
        optimizer.step()

        global_step += 1
        cur_nimg += B
        ema_flow.update(cur_nimg=cur_nimg, batch_size=B)  # assuming this mutates from current params

        loss_val = float(loss.detach().item())
        running_loss = loss_val if running_loss is None else (
            smooth_alpha * running_loss + (1 - smooth_alpha) * loss_val
        )

        pbar.set_postfix({
            "mse_loss": f"{loss_val:.4f}",
            "mse_loss_ema": f"{running_loss:.4f}",
            "steps": global_step
        })

    safe_tqdm_write(f"[epoch {ep+1}/{epoch}] steps={global_step:,} mse_loss={loss_val:.4f}, mse_loss_ema={running_loss:.4f}")

    if ep % 10 == 0:
        flow_model.save_pretrained(f"./checkpoints/flow_mnist")   # will overwrite; add suffix if you want snapshots
        ema_flow.save_pretrained(f"./checkpoints/flow_mnist")


print(f"Training done. Total steps: {global_step:,}, last loss: {loss_val:.4f}, EMA: {running_loss:.4f}")

In [ ]:
from rectified_flow.samplers import EulerSampler, SDESampler

ema_flow.apply_shadow()
model_inference = flow_model.eval()

rf_inference = RectifiedFlow(
    data_shape=data_shape,
    velocity_field=model_inference,
    device=device,
)

euler_sampler = EulerSampler(rf_inference, num_steps=200)
sde_sampler = SDESampler(rf_inference, num_steps=200, noise_scale=5, noise_decay_rate=0.)

In [ ]:
x_0 = torch.randn(30, *data_shape).to(device) * 0.5

x_1_euler = euler_sampler.sample_loop(x_0=x_0).trajectories[-1]
x_1_sde = sde_sampler.sample_loop(x_0=x_0).trajectories[-1]

print(x_1.shape)  # torch.Size([20, 1, 28, 28])

In [ ]:
from rectified_flow.utils import plot_cifar_results

plot_cifar_results(x_1_euler)
plot_cifar_results(x_1_sde)

## Train Conditional Generation

In [ ]:
model_type = "unet"    # "mlp" or "unet" or "dit"
from rectified_flow.models.enhanced_mlp import VarMLP
from rectified_flow.models.utils import EMAModel

if model_type == "unet":
    from rectified_flow.models.unet import SongUNet, SongUNetConfig
    config = SongUNetConfig(
        img_resolution = 28,
        in_channels = 1,                  # Number of color channels at input.
        out_channels = 1,                 # Number of color channels at output.
        label_dim = 0,                    # Number of class labels, 0 = unconditional.
        augment_dim = 0,                  # Augmentation label dimensionality, 0 = no augmentation.

        model_channels = 64,                # Base multiplier for the number of channels.
        channel_mult = [2, 2],              # Channel multipliers for each resolution.
        channel_mult_emb = 2,               # Multiplier for the dimensionality of the embedding vector.
        num_blocks = 2,                     # Number of residual blocks per resolution.
        attn_resolutions = [14],            # Resolutions at which to apply attention.
        dropout = 0.13,                     # Dropout probability of intermediate activations.
        label_dropout = 0.0,                # Dropout probability of class labels for classifier-free guidance.
        embedding_type = "positional",      # Timestep embedding type: 'positional' or 'fourier'.
        channel_mult_time = 1,              # Timestep embedding size: 1 for DDPM++, 2 for NCSN++.
        encoder_type = "standard",          # Encoder architecture: 'standard' or 'residual'.
        decoder_type = "standard",          # Decoder architecture: 'standard' or 'residual'.
        resample_filter = [1, 1]
    )
    flow_model = SongUNet(config)
    data_shape = (1, 28, 28)
elif model_type == "dit":
    raise NotImplementedError
else:
    raise ValueError(f"Unknown model type: {model_type}")

flow_model = flow_model.to(device)

print(f"Number of parameters in flow model: {sum(p.numel() for p in flow_model.parameters() if p.requires_grad):,}")

ema_flow = EMAModel(flow_model, ema_halflife_kimg=1.0, ema_rampup_ratio=0.05)

optimizer = torch.optim.AdamW(
    flow_model.parameters(),
    lr=5e-4, weight_decay=0.01,
    betas=(0.9, 0.95)
)

compiled_flow = torch.compile(flow_model, mode="reduce-overhead", fullgraph=False, dynamic=False)

compiled_flow.train()
optimizer.zero_grad()

In [ ]:
epoch = 200
cur_nimg = 0

rf_train = RectifiedFlow(
    data_shape=data_shape,
    velocity_field=compiled_flow,
    train_time_distribution="uniform",
    device=device,
)

try:
    from tqdm.auto import tqdm
except Exception:
    from tqdm import tqdm

def safe_tqdm_write(msg: str):
    try:
        tqdm.write(msg)
    except Exception:
        print(msg)

zero_to_none = True
grad_clip_norm = None

global_step = 0
running_loss = None
smooth_alpha = 0.99

# test model
with torch.no_grad():
    batch = next(iter(train_dataloader))
    x_1, c = batch
    x_1 = x_1.to(device, non_blocking=True).reshape(x_1.shape[0], *data_shape)
    c = c.to(device, non_blocking=True)
    c_onehot = torch.nn.functional.one_hot(c, num_classes=10).float()
    x_0 = torch.randn_like(x_1)
    t = rf_train.sample_train_time(x_1.shape[0]).to(device, non_blocking=True)
    # print(x_1.shape, x_0.shape, c.shape, t.shape)
    v_pred = flow_model(x_1, t, c_onehot)
    sq_err = (v_pred - (x_1 - x_0)).pow(2).sum(dim=1)
    loss = sq_err.mean()
    safe_tqdm_write(f"Initial test loss: {loss.item():.4f}")

In [ ]:
for ep in tqdm(range(epoch), desc="Epochs", position=0):
    pbar = tqdm(train_dataloader, desc=f"ep {ep+1}/{epoch}", leave=False, position=1)
    for step, batch in enumerate(pbar):
        optimizer.zero_grad(set_to_none=zero_to_none)

        x_1, c = batch
        c_onehot = torch.nn.functional.one_hot(c, num_classes=10).float()
        c_onehot = c_onehot.to(device, non_blocking=True)

        x_1 = x_1.to(device, non_blocking=True).reshape(x_1.shape[0], *data_shape)
        x_0 = torch.randn_like(x_1)
        t = rf_train.sample_train_time(x_1.shape[0]).to(device, non_blocking=True)

        # --- forward (bf16 autocast) ---
        with torch.autocast(device_type="cuda", dtype=torch.bfloat16):
            x_t, dot_x_t = rf_train.get_interpolation(x_0=x_0, x_1=x_1, t=t)
            v_pred = rf_train.get_velocity(x_t, t, class_labels=c_onehot)

        # --- pure MSE loss in fp32 for stability ---
        loss = torch.nn.functional.mse_loss(v_pred.float(), dot_x_t.float())
        mse_loss = float(loss.detach().item())

        loss.backward()

        if grad_clip_norm is not None:
            torch.nn.utils.clip_grad_norm_(rf_train.parameters(), grad_clip_norm)

        optimizer.step()

        global_step += 1
        cur_nimg += x_1.shape[0]
        ema_flow.update(cur_nimg=cur_nimg, batch_size=x_1.shape[0])

        loss_val = float(loss.detach().item())
        running_loss = loss_val if running_loss is None else (
            smooth_alpha * running_loss + (1 - smooth_alpha) * mse_loss
        )

        pbar.set_postfix({
            "mse_loss": f"{mse_loss:.4f}",
            "mse_loss_ema": f"{running_loss:.4f}",
            "steps": global_step
        })

    safe_tqdm_write(
        f"[epoch {ep+1}/{epoch}] steps={global_step:,} "
        f"mse_loss={mse_loss:.4f}, mse_loss_ema={running_loss:.4f}"
    )

    if ep % 10 == 0:
        flow_model.save_pretrained(f"./checkpoints/flow_mnist_unet_conditionalaa")
        ema_flow.save_pretrained(f"./checkpoints/flow_mnist_unet_conditionalaa")

print(f"Training done. Total steps: {global_step:,}, last loss: {loss_val:.4f}, EMA: {running_loss:.4f}")

### Class Conditional Generation without CFG

In [ ]:
from rectified_flow.samplers import EulerSampler, SDESampler

# flow_model = SongUNet.from_pretrained(f"./checkpoints/flow_mnist_unet_conditional", use_ema=False).to(device)

model_inference = flow_model.eval()

rf_inference = RectifiedFlow(
    data_shape=data_shape,
    velocity_field=model_inference,
    device=device,
)

# generate 100 samples for each class
class_labels = torch.arange(10, device=device)
class_labels = class_labels.repeat_interleave(10, dim=0)
class_onehot = torch.nn.functional.one_hot(class_labels, num_classes=10).float()

euler_sampler = EulerSampler(rf_inference, num_steps=50, num_samples=100)
sde_sampler = SDESampler(rf_inference, num_steps=50, noise_scale=50, noise_decay_rate=0., num_samples=100)

In [ ]:
x_1_euler = euler_sampler.sample_loop(seed=33, class_labels=class_onehot).trajectories[-1]
x_1_sde = sde_sampler.sample_loop(seed=33, class_labels=class_onehot).trajectories[-1]

print(x_1_euler.shape)  # torch.Size([20, 1, 28, 28])

In [ ]:
from rectified_flow.utils import plot_cifar_results

plot_cifar_results(x_1_euler)
plot_cifar_results(x_1_sde)

### Class Conditional Generation with CFG

In [ ]:
from rectified_flow.samplers import EulerSampler, SDESampler

flow_model = SongUNet.from_pretrained(f"./checkpoints/flow_mnist_unet_conditional", use_ema=True).to(device)

model_inference = flow_model.eval()

def velocity_with_cfg(x, t, class_labels=None, cfg_scale=3.5):
    assert class_labels is not None
    v_uncond = model_inference(x, t, torch.zeros_like(class_labels))
    v_cond = model_inference(x, t, class_labels)
    return v_uncond + cfg_scale * (v_cond - v_uncond)

rf_inference = RectifiedFlow(
    data_shape=data_shape,
    velocity_field=velocity_with_cfg,
    device=device,
)

# generate 100 samples for each class
class_labels = torch.arange(10, device=device)
class_labels = class_labels.repeat_interleave(10, dim=0)
class_onehot = torch.nn.functional.one_hot(class_labels, num_classes=10).float()

euler_sampler = EulerSampler(rf_inference, num_steps=50, num_samples=100)
sde_sampler = SDESampler(rf_inference, num_steps=50, noise_scale=5, noise_decay_rate=0., num_samples=100)

In [ ]:
x_1_euler = euler_sampler.sample_loop(seed=33, class_labels=class_onehot).trajectories[-1]
x_1_sde = sde_sampler.sample_loop(seed=33, class_labels=class_onehot).trajectories[-1]

print(x_1_euler.shape)  # torch.Size([20, 1, 28, 28])

In [ ]:
from rectified_flow.utils import plot_cifar_results

plot_cifar_results(x_1_euler)
plot_cifar_results(x_1_sde)